<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->




# **P1 - RA ÍMPAR**

In [ ]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Bruno Bosa Lopes, 20.00041-3' #@param {type:"string"}






In [ ]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

import pandas as pd

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [ ]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [ ]:
host_counts = df.groupby('host_id')['id'].count().reset_index()
host_counts = pd.merge(host_counts, df[['host_id', 'host_name']], on='host_id', how='left').drop_duplicates()
host_counts.rename(columns={'id':'propriedades'}, inplace=True)

top_3_hosts = host_counts.sort_values('propriedades', ascending=False).head(3)

print(top_3_hosts[['host_name', 'propriedades']])

        host_name  propriedades
734  Sonder (NYC)             7
403          Kara             4
621       Stanley             4


# Q2B. Preço nos bairros com mais imóveis

 Qual o preço médio dos imóveis, considerando somente os preços dos imóveis cujos bairros (`neighbourhood`) tenham mais de 20 imóveis sendo anunciados?  


In [ ]:
neighborhood_counts = df['neighbourhood'].value_counts()
neighborhoods_above20 = neighborhood_counts[neighborhood_counts > 20].index

df_above20 = df[df['neighbourhood'].isin(neighborhoods_above20)]
average_price = df_above20['price'].mean()

print(f"O preço médio dos imóveis em bairros com mais de 20 anúncios é: ${average_price:.2f}")


# print(f'O preço médio dos imóveis em bairros com mais de 20 anúncios é: USD {preco_medio:.2f}')

O preço médio dos imóveis em bairros com mais de 20 anúncios é: $151.42


# Q3B.  Brooklyn *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [ ]:
import numpy as np

# quartis
Q1 = df[df['neighbourhood_group'] == 'Brooklyn']['price'].quantile(0.25)
Q3 = df[df['neighbourhood_group'] == 'Brooklyn']['price'].quantile(0.75)
IQR = Q3 - Q1

#lower e upper
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

df_outliers = df[(df['neighbourhood_group'] == 'Brooklyn') & ((df['price'] < lower) | (df['price'] > upper))]
percentual_outliers = (len(df_outliers) / len(df[df['neighbourhood_group'] == 'Brooklyn'])) * 100

print( f'Percentuais de outliers de preços, Brooklyn: {percentual_outliers:.2f} %')


Percentuais de outliers de preços, Brooklyn: 5.76 %


In [ ]:
df[df['neighbourhood_group'] == 'Brooklyn']['price'].sort_values().describe()

count    1251.000000
mean      128.171863
std       136.282636
min         0.000000
25%        64.000000
50%        98.000000
75%       150.000000
max      1800.000000
Name: price, dtype: float64

# Q4B. Scores

Nesta tarefa você deve atribuir a cada imóvel um valor de *score* (`df['score']`) baseado nas seguintes regras:

1. Imóveis com `availability_365` > 90 recebem +10 pontos enquanto imóveis com `availability_365` < 5 recebem -5 pontos;
2. Imóveis com o número de reviews >= 10 recebem +5 pontos para cada 10 reviews (por ex. se número de reviews = 20, são atribuídos +10 pontos). Se número de reviews = 0 o imóvel recebe -5 pontos;
3. Imóveis com preço abaixo da média de seu `neighbourhood`	recebem +5 pontos.

Implemente empregando `apply()` ou um laço de programa, como preferir.

In [ ]:
media_bairro = df.groupby('neighbourhood')['price'].mean()

df['score'] = 0
#iterrows
for index, row in df.iterrows():
  #1
  if row['availability_365'] > 90:
    df.loc[index, 'score'] += 10
  elif row['availability_365'] < 5:
    df.loc[index, 'score'] -= 5
  #2
  if row['number_of_reviews'] >= 10:
    df.loc[index, 'score'] += (row['number_of_reviews'] // 10) * 5
  elif row['number_of_reviews'] == 0:
    df.loc[index, 'score'] -= 5
  #3
  if row['price'] < media_bairro[row['neighbourhood']]:
    df.loc[index, 'score'] += 5



print(f'A soma dos valores de score é: ', df['score'].sum())


A soma dos valores de score é:  10955


# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [ ]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [ ]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')

#trazendo tax e fines para mesma linha, como coluna nova
df_taxes = df_taxes.pivot(index='id', columns='type', values='value').reset_index()
df_taxes.columns = ['id', 'fines', 'taxes']
#merge + Nan
df = pd.merge(df, df_taxes, on='id', how='left')
df[['taxes', 'fines']] = df[['taxes','fines']].fillna(0)

result = df.groupby('neighbourhood_group').agg({'taxes': 'mean', 'fines': 'mean'})
result


taxes       fines
neighbourhood_group                        
Bronx                808.500000  120.150000
Brooklyn             798.457576  292.554545
Manhattan            855.029891  302.997283
Queens               793.987342  318.683544
Staten Island        992.142857  498.285714

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [ ]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,26,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]